### Mejorar la eficacia de la red 

Ahora, cambiaremos ciertos parámetros de la red (optimizador, capas, función de activación) trabajada en keras para tratar de obtener mayor eficacia en nuestra red. A cada configuración le llamaremos un experimento.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dataset = mnist.load_data() #descargar la base de datos MNIST

In [32]:
!pip install mlflow --quiet

In [5]:
import mlflow
import mlflow.tensorflow
mlflow.tensorflow.autolog()

In [21]:
import mlflow.keras
import os
from getpass import getpass

#vinculamos la cuenta de dagshub con mlflow
os.environ['MLFLOW_TRACKING_USERNAME'] = "mseguracarrillo7"
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass("Enter your DAGsHub access token or password: ")
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mseguracarrillo7/Red-densa"
mlflow.set_tracking_uri("https://dagshub.com/mseguracarrillo7/Red-densa.mlflow")

Enter your DAGsHub access token or password:  ········


In [11]:
#datos de prueba y entrenamiento
(x_train, y_train), (x_test, y_test) = dataset 

In [13]:
num_classes=10  #one hot encouding para y 
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)
#normalizamos las imágenes a valores entre 0 y 1 
x_trainv = x_train.reshape(60000, 784)
x_testv = x_test.reshape(10000, 784)
x_trainv = x_trainv.astype('float32')
x_testv = x_testv.astype('float32')

x_trainv /= 255.  # x_trainv = x_trainv/255
x_testv /= 255.

#### Experimento 1

Añadiremos más neuronas en nuevas capas, se va a utilizar la función sigmoide y en la última capa una softmax.
Usaremos el optimizador Adam y la función de costo "categorical cross entropy".

In [22]:
def model_exp1():
    
    model = Sequential()
    model.add(Dense(512, activation='sigmoid', input_shape=(784,))) 
    model.add(Dense(128, activation='sigmoid', input_shape=(784,)))
    model.add(Dense(64, activation='sigmoid', input_shape=(784,)))
    model.add(Dense(num_classes, activation='softmax')) #aplicamos softmax en la capa de salida

    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
    return model

In [32]:
import mlflow
import mlflow.keras
import matplotlib.pyplot as plt

# conectamos con mlflow
with mlflow.start_run(run_name="Experimento 1 - Más neuronas"):
    model = model_exp1()
    history = model.fit(x_trainv, y_trainc, 
                        validation_data=(x_testv, y_testc),
                        epochs=10, batch_size=32, verbose=1)

    # Registrar modelo entrenado
    model.save("Modelo1.keras")
    mlflow.log_artifact("modelo1.keras")

    # Registrar hiperparámetros
    mlflow.log_params({
        "epochs": 10,
        "batch_size": 32,
        "optimizer": "adam"
    })

    # Registrar métricas última capa [-1]
    mlflow.log_metrics({
        "train_accuracy": history.history["accuracy"][-1],
        "val_accuracy": history.history["val_accuracy"][-1]
    })

    # Registrar métricas por época en mlflow
    for epoch in range(len(history.history["accuracy"])):
        mlflow.log_metric("train_accuracy_epoch", history.history["accuracy"][epoch], step=epoch)
        mlflow.log_metric("val_accuracy_epoch", history.history["val_accuracy"][epoch], step=epoch)
        mlflow.log_metric("train_loss_epoch", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss_epoch", history.history["val_loss"][epoch], step=epoch)

    # Gráfica de accuracy
    plt.figure()
    plt.plot(history.history["accuracy"], label="Entrenamiento")
    plt.plot(history.history["val_accuracy"], label="Validación")
    plt.xlabel("Época")
    plt.ylabel("Exactitud")
    plt.legend()
    plt.title("Curva de Accuracy")
    plt.savefig("accuracy_curve.png")
    plt.close()

    # Gráfica de loss
    plt.figure()
    plt.plot(history.history["loss"], label="Entrenamiento")
    plt.plot(history.history["val_loss"], label="Validación")
    plt.xlabel("Época")
    plt.ylabel("Pérdida")
    plt.legend()
    plt.title("Curva de Pérdida")
    plt.savefig("loss_curve.png")
    plt.close()

    # Subir imágenes a MLflow/Dagshub
    mlflow.log_artifact("accuracy_curve.png")
    mlflow.log_artifact("loss_curve.png")


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 36s 16ms/step - accuracy: 0.8891 - loss: 0.4126 - val_accuracy: 0.9486 - val_loss: 0.1748
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9591 - loss: 0.1399 - val_accuracy: 0.9669 - val_loss: 0.1105
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - accuracy: 0.9725 - loss: 0.0917 - val_accuracy: 0.9699 - val_loss: 0.0959
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9800 - loss: 0.0640 - val_accuracy: 0.9739 - val_loss: 0.0804
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9858 - loss: 0.0474 - val_accuracy: 0.9722 - val_loss: 0.0915
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 29s 16ms/step - accuracy: 0.9890 - loss: 0.0352 - val_accuracy: 0.9784 - val_loss: 0.0714
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/step - accuracy: 0.9917 - loss: 0.0276 - val_accuracy: 0.9783 - val_loss: 0.0795
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9934 -

2025/09/12 17:52:59 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}


🏃 View run Experimento 1 - Más neuronas at: https://dagshub.com/mseguracarrillo7/Red-densa.mlflow/#/experiments/0/runs/b18b1ca4bb744bee852584dea448cc7c
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Red-densa.mlflow/#/experiments/0


#### Experimento 2

En este modelo, también agregaremos capas con neuronas, pero usaremos como función de activuación a $relu$, mantendremos categorical cross entropy y también el optimizador Adam, pero ahora con un learning rate de 0.0005.

In [15]:
optimizer = Adam(learning_rate=0.0005)

In [17]:
# Red más ancha y con función de acttivación 'relu' , la última sigue siendo softmax
def model_exp2():
    model2 = Sequential()
    model2.add(Dense(512, activation='relu', input_shape=(784,))) 
    model2.add(Dense(128, activation='relu', input_shape=(784,)))
    model2.add(Dense(128, activation='relu', input_shape=(784,)))
    model2.add(Dense(64, activation='relu', input_shape=(784,)))
    model2.add(Dense(num_classes, activation='softmax')) #aplicamos softmax en la capa de salida

    model2.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])
    return model2

In [23]:
with mlflow.start_run(run_name="Experimento 2 - Relu"):
    model2 = model_exp2()
    history = model2.fit(x_trainv, y_trainc, 
                        validation_data=(x_testv, y_testc),
                        epochs=10, batch_size=32, verbose=1)

    # Registrar modelo entrenado
    model2.save("Modelo2.keras")
    mlflow.log_artifact("modelo2.keras")

    # Registrar hiperparámetros
    mlflow.log_params({
        "epochs": 10,
        "batch_size": 32,
        "optimizer": "adam"
    })

    # Registrar métricas última capa [-1]
    mlflow.log_metrics({
        "train_accuracy": history.history["accuracy"][-1],
        "val_accuracy": history.history["val_accuracy"][-1]
    })

    # Registrar métricas por época en mlflow
    for epoch in range(len(history.history["accuracy"])):
        mlflow.log_metric("train_accuracy_epoch", history.history["accuracy"][epoch], step=epoch)
        mlflow.log_metric("val_accuracy_epoch", history.history["val_accuracy"][epoch], step=epoch)
        mlflow.log_metric("train_loss_epoch", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss_epoch", history.history["val_loss"][epoch], step=epoch)
    

C:\Users\Moises\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 15ms/step - accuracy: 0.9353 - loss: 0.2120 - val_accuracy: 0.9666 - val_loss: 0.1106
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.9717 - loss: 0.0944 - val_accuracy: 0.9702 - val_loss: 0.1010
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - accuracy: 0.9792 - loss: 0.0673 - val_accuracy: 0.9743 - val_loss: 0.0895
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9837 - loss: 0.0539 - val_accuracy: 0.9778 - val_loss: 0.0841
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 29s 16ms/step - accuracy: 0.9863 - loss: 0.0442 - val_accuracy: 0.9773 - val_loss: 0.0821
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - accuracy: 0.9888 - loss: 0.0350 - val_accuracy: 0.9792 - val_loss: 0.0778
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 39s 14ms/step - accuracy: 0.9902 - loss: 0.0313 - val_accuracy: 0.9789 - val_loss: 0.0827
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9915 -

2025/09/12 19:08:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}


🏃 View run Experimento 2 - Relu at: https://dagshub.com/mseguracarrillo7/Red-densa.mlflow/#/experiments/0/runs/dd90f65b567741e196da39ce9af5d9db
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Red-densa.mlflow/#/experiments/0


In [25]:
test_loss, test_acc = model2.evaluate(x_testv, y_testc, verbose=1)
print(f"Exactitud en el conjunto de prueba: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9812 - loss: 0.0821
Exactitud en el conjunto de prueba: 0.9812
Valor de la función de pérdida: 0.0821


#### Experimento 3

Para este experimento usaré la función de activación $tanh$ y cambiaré el optimizador por $SGD$. También aumentaré el tamaño de los mini batches y el número de épocas.

In [32]:
def model_exp3():
    model3 = Sequential()
    model3.add(Dense(512, activation='tanh', input_shape=(784,))) 
    model3.add(Dense(128, activation='tanh', input_shape=(784,)))
    model3.add(Dense(128, activation='tanh', input_shape=(784,)))
    model3.add(Dense(64, activation='tanh', input_shape=(784,)))
    model3.add(Dense(num_classes, activation='softmax')) #aplicamos softmax en la capa de salida

    model3.compile(optimizer=SGD(learning_rate=0.01),loss="categorical_crossentropy",metrics=["accuracy"])
    return model3

In [34]:
with mlflow.start_run(run_name="Experimento 3 - Tanh"):
    model3 = model_exp3()
    history = model3.fit(x_trainv, y_trainc, 
                        validation_data=(x_testv, y_testc),
                        epochs=30, batch_size=128, verbose=1)

    # Registrar modelo entrenado
    model3.save("Modelo3.keras")
    mlflow.log_artifact("modelo3.keras")

    # Registrar hiperparámetros
    mlflow.log_params({
        "epochs": 30,
        "batch_size": 128,
        "optimizer": SGD(learning_rate=0.01)
    })

    # Registrar métricas última capa [-1]
    mlflow.log_metrics({
        "train_accuracy": history.history["accuracy"][-1],
        "val_accuracy": history.history["val_accuracy"][-1]
    })

    # Registrar métricas por época en mlflow
    for epoch in range(len(history.history["accuracy"])):
        mlflow.log_metric("train_accuracy_epoch", history.history["accuracy"][epoch], step=epoch)
        mlflow.log_metric("val_accuracy_epoch", history.history["val_accuracy"][epoch], step=epoch)
        mlflow.log_metric("train_loss_epoch", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss_epoch", history.history["val_loss"][epoch], step=epoch)
    

Epoch 1/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.7938 - loss: 0.8535 - val_accuracy: 0.8838 - val_loss: 0.4706
Epoch 2/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.8918 - loss: 0.4166 - val_accuracy: 0.9064 - val_loss: 0.3544
Epoch 3/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.9064 - loss: 0.3418 - val_accuracy: 0.9137 - val_loss: 0.3098
Epoch 4/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.9156 - loss: 0.3041 - val_accuracy: 0.9206 - val_loss: 0.2833
Epoch 5/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9217 - loss: 0.2795 - val_accuracy: 0.9262 - val_loss: 0.2639
Epoch 6/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9255 - loss: 0.2605 - val_accuracy: 0.9294 - val_loss: 0.2465
Epoch 7/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.9302 - loss: 0.2448 - val_accuracy: 0.9312 - val_loss: 0.2357
Epoch 8/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9334 - loss: 0.2312 - val_ac

2025/09/12 19:26:53 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}


🏃 View run Experimento 3 - Tanh at: https://dagshub.com/mseguracarrillo7/Red-densa.mlflow/#/experiments/0/runs/41a8a5618076443594c1c2c9e771e17a
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Red-densa.mlflow/#/experiments/0


In [35]:
test_loss, test_acc = model3.evaluate(x_testv, y_testc, verbose=1)
print(f"Exactitud en el conjunto de prueba: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9673 - loss: 0.1072
Exactitud en el conjunto de prueba: 0.9673


#### Experimento 4

Para la útima configuración utilizaré la función de activación $selu$, el optimizador Adam y usaré el número de épocas y tamaño de mini batches del experimento 3, agregaré otra capa de neuronas.

In [48]:
def model_exp4():
    model4 = Sequential()
    model4.add(Dense(512, activation='selu', input_shape=(784,))) 
    model4.add(Dense(128, activation='selu', input_shape=(784,)))
    model4.add(Dense(128, activation='selu', input_shape=(784,)))
    model4.add(Dense(64, activation='selu', input_shape=(784,)))
    model4.add(Dense(64, activation='selu', input_shape=(784,)))
    model4.add(Dense(num_classes, activation='softmax')) #aplicamos softmax en la capa de salida

    model4.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
    return model4

In [50]:
with mlflow.start_run(run_name="Experimento 4 - Selu"):
    model4 = model_exp4()
    history = model4.fit(x_trainv, y_trainc, 
                        validation_data=(x_testv, y_testc),
                        epochs=30, batch_size=128, verbose=1)

    # Registrar modelo entrenado
    model4.save("Modelo4.keras")
    mlflow.log_artifact("modelo4.keras")

    # Registrar hiperparámetros
    mlflow.log_params({
        "epochs": 30,
        "batch_size": 128,
        "optimizer": "adam"
    })

    # Registrar métricas última capa [-1]
    mlflow.log_metrics({
        "train_accuracy": history.history["accuracy"][-1],
        "val_accuracy": history.history["val_accuracy"][-1]
    })

    # Registrar métricas por época en mlflow
    for epoch in range(len(history.history["accuracy"])):
        mlflow.log_metric("train_accuracy_epoch", history.history["accuracy"][epoch], step=epoch)
        mlflow.log_metric("val_accuracy_epoch", history.history["val_accuracy"][epoch], step=epoch)
        mlflow.log_metric("train_loss_epoch", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss_epoch", history.history["val_loss"][epoch], step=epoch)
    

Epoch 1/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - accuracy: 0.9166 - loss: 0.2765 - val_accuracy: 0.9485 - val_loss: 0.1633
Epoch 2/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9612 - loss: 0.1269 - val_accuracy: 0.9655 - val_loss: 0.1122
Epoch 3/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9690 - loss: 0.0977 - val_accuracy: 0.9606 - val_loss: 0.1270
Epoch 4/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.9748 - loss: 0.0783 - val_accuracy: 0.9682 - val_loss: 0.1036
Epoch 5/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.9796 - loss: 0.0649 - val_accuracy: 0.9711 - val_loss: 0.0972
Epoch 6/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.9806 - loss: 0.0603 - val_accuracy: 0.9754 - val_loss: 0.0939
Epoch 7/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9830 - loss: 0.0514 - val_accuracy: 0.9703 - val_loss: 0.1013
Epoch 8/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9846 - loss: 0.0462 - v

2025/09/12 19:46:51 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}


🏃 View run Experimento 4 - Selu at: https://dagshub.com/mseguracarrillo7/Red-densa.mlflow/#/experiments/0/runs/fe3498f37e8c4179a012f11623b59060
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Red-densa.mlflow/#/experiments/0


In [51]:
test_loss, test_acc = model4.evaluate(x_testv, y_testc, verbose=1)
print(f"Exactitud en el conjunto de prueba: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9769 - loss: 0.1110
Exactitud en el conjunto de prueba: 0.9769
